In [1260]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import pprint
import sklearn
from scipy.stats import ttest_ind



### Load Dataset

In [1261]:
df = pd.read_csv('./master.csv')
df.head(10)

,Unnamed: 0,Mouser Part Number,Mfr Part Number,Mfr.,Datasheet,Availability,Pricing,RoHS,Lifecycle,Product Detail,IP Rating,Product,Contact Gender,Termination Style,UID
0,0,651-1404642,"=""1404642""",Phoenix Contact,https://www.mouser.com/datasheet/2/324/3/14046...,656 In Stock,"=""$33.09""",RoHS Compliant By Exemption,NaN,https://www.mouser.com/ProductDetail/Phoenix-C...,IP67,Connectors,Socket (Female),Screw,1
1,1,654-MB7CKN0900-S1,MB7CKN0900-S1,Amphenol SINE Systems,https://www.amphenol-sine.com/pdf/datasheet/MB...,89 In Stock,"=""$25.36""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP67,Connectors,Without Socket Contacts,Crimp,2
2,2,523-T3636-001,T 3636 001,Amphenol Tuchel,https://www.amphenol-sine.com/pdf/datasheet/T3...,221 In Stock,"=""$20.70""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP40,Connectors,Socket (Female),Solder,3
3,3,571-5-2271137-2,5-2271137-2,TE Connectivity,https://www.mouser.com/datasheet/2/418/7/ENG_C...,93 In Stock,"=""$37.10""",RoHS Compliant By Exemption,NaN,https://www.mouser.com/ProductDetail/TE-Connec...,IP68,Connectors,Socket (Female),Solder,4
4,4,651-1441970,"=""1441970""",Phoenix Contact,https://www.mouser.com/datasheet/2/324/3/14419...,788 In Stock,"=""$30.77""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Phoenix-C...,IP67,Inserts,Socket (Female),Through Hole,5
5,5,523-C01620D00320012,C016 20D003 200 12,Amphenol Tuchel,https://www.amphenol-sine.com/pdf/datasheet/C0...,315 In Stock,"=""$20.53""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP67,Connectors,Socket (Female),Screw,6
6,6,523-C09111D0080024U,C09111D0080024U,Amphenol Tuchel,https://www.amphenol-sine.com/pdf/datasheet/C0...,264 In Stock,"=""$13.17""",NaN,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP68,Connectors,Socket (Female),-,7
7,7,654-RT0W7106PNH,RT0W7106PNH,Amphenol SINE Systems,https://www.amphenol-sine.com/pdf/datasheet/RT...,113 In Stock,"=""$15.22""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP67,Connectors,Socket (Female),-,8
8,8,523-T3638-000,T 3638 000,Amphenol Tuchel,https://www.amphenol-sine.com/pdf/datasheet/T3...,149 In Stock,"=""$17.64""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP40,Connectors,Socket (Female),Solder,9
9,9,654-RT00104SNH,RT00104SNH,Amphenol SINE Systems,https://www.amphenol-sine.com/pdf/datasheet/RT...,493 In Stock,"=""$9.18""",RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,IP67,Connectors,Socket (Female),-,10


In [1262]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8150 entries, 0 to 8149
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          8150 non-null   int64 
 1   Mouser Part Number  8150 non-null   object
 2   Mfr Part Number     8150 non-null   object
 3   Mfr.                8150 non-null   object
 4   Datasheet           7964 non-null   object
 5   Availability        7902 non-null   object
 6   Pricing             8133 non-null   object
 7   RoHS                6781 non-null   object
 8   Lifecycle           1778 non-null   object
 9   Product Detail      8150 non-null   object
 10  IP Rating           7094 non-null   object
 11  Product             6991 non-null   object
 12  Contact Gender      7267 non-null   object
 13  Termination Style   7555 non-null   object
 14  UID                 8150 non-null   int64 
dtypes: int64(2), object(13)
memory usage: 955.2+ KB


### Exploring  unique values

In [1263]:
pp = pprint.PrettyPrinter(indent=4)

def getUniqueValues(feature, a):
    listFeature = a[feature].unique()
    # pp.pprint(feature, " unique values: ", listFeature)
    # pp.pprint(feature, " nunique = ", len(listFeature))
    print(len(listFeature), " Unique values of: ", feature)
    pp.pprint(listFeature)
    print("\n")

##Mfr. values --45 unique manufacturers
getUniqueValues("Mfr.", df)

##Availability values --"xx" In Stock, Non-stocked Lead Time "x" weeks, 'x' Stock Available
##                       Lead Time "x" weeks, "x" Factory Stock Available, 
df_Availability = df.dropna(subset=['Availability'])
df_Availability_Not_In_Stock = df_Availability[~df_Availability['Availability'].str.contains('In Stock')]
getUniqueValues("Availability", df_Availability_Not_In_Stock)

##Pricing values --all valid values have prices attached in format '=$xx.xx'
df_Pricing = df.dropna(subset=['Pricing'])
df_Pricing_No_Price = df_Pricing[df_Pricing['Pricing'].str.contains('$')]

##RoHS values --Compliant, Complaint By Exemption, nan, Not Applicable, No, In Transition
getUniqueValues('RoHS', df)

##Lifecycle values --nan, 'New Product', 'New At Mouser', 'End of Life',
##       'Verify Status with Factory', 'Preliminary Release'
getUniqueValues('Lifecycle', df)

##IP Rating values -- 36 different IP Ratings
getUniqueValues("IP Rating", df)

##Product values -- 32 different product categories: "Connectors', 'Inserts', 'Accessories', 'Couplers', 'Plugs', 'nan'
getUniqueValues("Product", df)

##Contact Gender values -- 19 different Contact Gender categories: "Pin (Male)", "Socket (Female)", 'nan'
getUniqueValues("Contact Gender", df)

##Termination Style --28 unique categories: 'Screw', 'Crimp', 'Solder', 'nan'
getUniqueValues("Termination Style", df)

45  Unique values of:  Mfr.
array(['Phoenix Contact', 'Amphenol SINE Systems', 'Amphenol Tuchel',
       'TE Connectivity', 'Bulgin', 'Molex', 'Amphenol LTW',
       'Hirose Connector', 'Amphenol Air LB Germany',
       'Amphenol Aerospace', 'TE Connectivity / DEUTSCH',
       'Amphenol Industrial', 'LEMO', 'ODU', 'Amphenol Socapex',
       'Amphenol Pcd', 'Souriau', 'Switchcraft', 'ITT Cannon',
       'Amphenol CONEC', 'HARTING', 'Omron Automation and Safety',
       'Amphenol Positronic', 'TE Connectivity / Intercontec',
       'Conxall / Switchcraft', 'NorComp', 'CUI Devices',
       'Stewart Connector / Bel', 'Murrelektronik',
       'Amphenol Commercial Products', 'Lumberg Automation',
       'Amphenol Alden Products', 'TE Connectivity / AMP', 'Amphenol LTD',
       'Amphenol', 'Glenair', 'Weidmuller', 'Banner Engineering',
       'TE Connectivity / Raychem', 'Wurth Elektronik', 'JAE Electronics',
       'TE Connectivity / ERNI', 'TE Connectivity / Polamco',
       'Amphenol Canad

In [1264]:
df.isnull().sum()

Unnamed: 0               0
Mouser Part Number       0
Mfr Part Number          0
Mfr.                     0
Datasheet              186
Availability           248
Pricing                 17
RoHS                  1369
Lifecycle             6372
Product Detail           0
IP Rating             1056
Product               1159
Contact Gender         883
Termination Style      595
UID                      0
dtype: int64

### **Observations:**
We note that there are 14 characteristics, with a majority of them being of object type, which typically includes textual data. Hence, we will need to use word vectorization to convert them to numerical values to build our classification model. 

**Missing values:**

* `Pricing` has significantly small number of missing values (17)
* `Lifecycle` has significantly large number of missing values (6372)
* Handle missing values for `Lifecycle`, `RoHS`, `IP Rating`, `Product`
* Other features can be filled in simply 
* 
* `Lifecycle`: **Nominal**, Mode = New Product (92%)
* `RoHS`: **Nominal/Ordinal**, Mode = RoHS Compliant (78%), RoHS Compliant By Exemption (18%)
* `IP Rating`: **Ordinal**, Mode = IP67(37%), IP68(15%), IP50(10%)
* `Product`: **Nominal**, Mode = 'Connectors'(57%), Receptacles(14%), Accessories(8%), Plugs(6%)

**Categorical Features:**

Some columns, such as `Manufacturer`, `Product`, `Contact Gender`, and `Termination Style`, contain categorical data.
These categorical features will need to be encoded before being used in the model.
* `Lifecycle`: **Nominal**
    * EOL Products reaching end of lifecycle may have different characteristics compared ot new
    * New vs EOL: change in manufacturing processes/materials/design revisions
    * New vs Mature: Undergo different levels of testing/level of reliability
    * Updates and Revisions: **Preliminary Release** subject to revisions *(might be insignificant %)*
* `RoHS`: **Nominal/Ordinal** Restriciton of Hazardous Substances(lead, mercury)
    * Presence/absence of **hazardous** subsatnces -> prone to corrosion can affect durability/reliability -> IP Rating
    * Comply with RoHS regulations -> stricter QC -> consistent performance and better IP
* `Product`: **Nominal**
    * Physical Characteristics: Varying physical characteristics that affect IP
    * **Connectors** and **cable clamp/backshells** typically have openings that may require additional protection against environmental elements compared to **plugs**/**housings**
    * Design Usage/Application: **Gaskets**/**Seals** inherently prioritize IP to seal against water/dust
* `Contact Gender`: **Nominal/Ordinal**, Plug vs Socket: Female = receptacle receiving male connectors
    * Design of female connectors may incorporate additional sealing features
* `Termination Style`: **Nominal**
    * Design: **Crimp**/**Cramp**/**Solder** terminations -> more secure/stable/better protection
    
**Duplicates:**
* Some entries have have same `UIDs`(same PDF datasheet file)
* But have different `Pricing`, `Availability`, similar `Part Number`
* Conclusion: Subsets of same UID/Component with slightly different variations
* Investigate if variations of component have same IP rating

**Unique values:**
* `RoHS`: 5
* `Lifecycle`: 5
* `IP Rating`: 35
* `Product`: 31
* `Contact Gender`: 18
* `Termination Style`: 27


##### Decided to remove missing values for `Pricing`

In [1265]:
# Remove missing values for `Pricing` because insignificant(17 entries removed)
df1 = df.dropna(subset=['Pricing'])

#### Optional Investigation/Exploration(Deferred)
1. Investigate IP Ratings of variations of same component
2. Correlation between IP Ratings
3. Group By

In [1266]:
df.duplicated(subset=['Mouser Part Number']).sum()
duplicates = df[df.duplicated(subset=['UID'])]
# print(duplicates.to_string())
df["UID"].nunique() #3610 unique UIDs
df["Mfr Part Number"].nunique() #5718 unique part numbers

5718

#### Preliminary data cleaning
* `Pricing`: Standardized by removing irrelevant string parts and ocnverting to float
* `Contact Gender`: As certain entries had multiple Contact Genders in 1 entry separated by a comma, I initially wanted to create 3(max) genders to encode, but realised it was too insiginificant and abandoned the idea

In [1267]:

pd.options.mode.chained_assignment = None  # default='warn'

# Standardize Pricing, IP Rating
df1['Pricing'] = df1['Pricing'].str.strip('="$')

df1['Pricing'] = df1['Pricing'].apply(lambda x: x.replace(",", ""))
df1['Pricing'] = df1['Pricing'].apply(lambda x: float(x))
# Cleaning Contact Gender
df1['Contact_1'] = df1['Contact Gender'].str.split(',').str.get(0).str.strip()
df1['Contact_2'] = df1['Contact Gender'].str.split(',').str.get(1).str.strip()
df1['Contact_3'] = df1['Contact Gender'].str.split(',').str.get(2).str.strip()

# Replace empty strings with NaN
df1['Contact_2'].replace('', np.nan, inplace=True)
df1['Contact_3'].replace('', np.nan, inplace=True)
df1["Contact_1"].replace('-', np.nan, inplace=True)

# Replace additional columns with NaN where applicable
df1.loc[df1['Contact_2'].isnull(), 'Contact_2'] = np.nan
df1.loc[df1['Contact_3'].isnull(), 'Contact_3'] = np.nan
listCG1 = set(df1['Contact_1'].unique().flatten())
listCG2 = set(df1['Contact_2'].unique().flatten())
listCG3 = set(df1['Contact_3'].unique().flatten())
listCG_unique = listCG3.union(listCG1, listCG3)
len(listCG_unique) #11 unique Contact Gender values




11

In [1268]:
#Create a dataframe consisting of hot-encoded CG values
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
# CG_OH_encoding = pd.get_dummies(df1['Department']).drop(columns='Finance')
encoder = OneHotEncoder()
categorical_columns = ['Contact_1', 'Contact_2','Contact_3']
one_hot_encoded = encoder.fit_transform(df1[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded)
one_hot_df

one_hot_encoded_data = pd.get_dummies(df1, columns = ['Contact_1','Contact_2','Contact_3'])
one_hot_encoded_data


,Unnamed: 0,Mouser Part Number,Mfr Part Number,Mfr.,Datasheet,Availability,Pricing,RoHS,Lifecycle,Product Detail,...,Contact_1_Socket,Contact_1_Socket (Female),Contact_1_With Pin Contacts,Contact_1_Without Pin Contacts,Contact_1_Without Pin or Socket Contacts,Contact_1_Without Socket Contacts,Contact_2_Pin (Male),Contact_2_Socket (Female),Contact_3_Pin (Male),Contact_3_Socket (Female)
0,0,651-1404642,"=""1404642""",Phoenix Contact,https://www.mouser.com/datasheet/2/324/3/14046...,656 In Stock,33.09,RoHS Compliant By Exemption,NaN,https://www.mouser.com/ProductDetail/Phoenix-C...,...,0,1,0,0,0,0,0,0,0,0
1,1,654-MB7CKN0900-S1,MB7CKN0900-S1,Amphenol SINE Systems,https://www.amphenol-sine.com/pdf/datasheet/MB...,89 In Stock,25.36,RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,...,0,0,0,0,0,1,0,0,0,0
2,2,523-T3636-001,T 3636 001,Amphenol Tuchel,https://www.amphenol-sine.com/pdf/datasheet/T3...,221 In Stock,20.70,RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Amphenol-...,...,0,1,0,0,0,0,0,0,0,0
3,3,571-5-2271137-2,5-2271137-2,TE Connectivity,https://www.mouser.com/datasheet/2/418/7/ENG_C...,93 In Stock,37.10,RoHS Compliant By Exemption,NaN,https://www.mouser.com/ProductDetail/TE-Connec...,...,0,1,0,0,0,0,0,0,0,0
4,4,651-1441970,"=""1441970""",Phoenix Contact,https://www.mouser.com/datasheet/2/324/3/14419...,788 In Stock,30.77,RoHS Compliant,NaN,https://www.mouser.com/ProductDetail/Phoenix-C...,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8145,8145,649-CL142001,CL142001,Souriau,https://www.mouser.com/datasheet/2/379/clipper...,98 In Stock,17.90,NaN,NaN,https://www.mouser.com/ProductDetail/Souriau/C...,...,0,0,0,0,0,0,0,0,0,0
8146,8146,649-HAUTS0V128PRA300,HAUTS0V128PRA300,Souriau,https://www.mouser.com/datasheet/2/379/UTS_Ser...,1 In Stock,87.30,NaN,NaN,https://www.mouser.com/ProductDetail/Souriau/H...,...,0,0,0,0,0,0,0,0,0,0
8147,8147,649-JBXFH2C16MSSDSMR,JBXFH2C16MSSDSMR,Souriau,https://ipaper.ipapercms.dk/SOURIAU/JBXSeries/,17 In Stock,101.41,NaN,NaN,https://www.mouser.com/ProductDetail/Souriau/J...,...,0,0,0,0,0,0,0,0,0,0
8148,8148,649-JBXEA1G10MSSDS,JBXEA1G10MSSDS,Souriau,https://ipaper.ipapercms.dk/SOURIAU/JBXSeries/,25 In Stock,32.88,NaN,NaN,https://www.mouser.com/ProductDetail/Souriau/J...,...,0,0,0,0,0,0,0,0,0,0


#### Dropping of features that do not have a high correlation
* Decision on columns were primarily based on intuition as there was not enough time to check correlation, explore data visualisations and plots to derive potential insights and guidance

In [1269]:
df1.drop('Product Detail', axis=1, inplace=True)
df1.drop('Mouser Part Number', axis =1, inplace=True)
df1.drop('Unnamed: 0', axis =1, inplace=True)
df1.drop('Datasheet', axis =1, inplace=True)
df1.drop('Availability', axis =1, inplace=True)


#### IP Rating Standardization
* **Made the decision to separate IP Rating into two features corresponding to the different digits of Dust and Water**
* Standardized String processing, converted the data to numerical for ordering
* Took liberties with "K" Values, "X" values as seen in the hashmap\
69X -> 6,10
69K -> 6, 10
6k9k -> 6, 10
X7 -> na, 7
X9K -> na, 9k
X8 -> na, 8

In [1270]:
IP_list = ['IP67', 'IP40', 'IP68', 'IP65', 'IP65, IP67', 'IP67, IP68',\
       'IP66, IP67', 'IP69K, IP67', 'IP69X', 'IP67, IP69K', \
       'IP50', 'IP61', 'IP6K8, IP6K9K', 'IP67, 68', 'IPX7', 'IP66, IP68',\
       'IP65, IP67, IP69K', 'IP66', 'IP65, IP67, IP69, IP69K',\
       'IP65, IP68', 'IP65, IP68, IP69K', 'IP69', 'IP64', 'IPX8, IPX9K',\
       'IPX8', 'IP66, IP67, IP68', 'IPX9K', 'IP69K', 'IP54', 'IP20',\
       'IP67, IP69, IPX9K', 'IP67, IP68, IP69K', 'IP68, IP69K',\
       'IP68/69K']
sorted(IP_list)

#Note: assume baseline IP_Rating of X not tested as zero
IP_1 = {'X': 0, "0": 0, "1": 1, "2":2, "3":3, "4":4, "5":5, "5K": 5, "6":6, "6K": 6}
IP_2 = {'X': 0, "0":0, "1": 1, "2":2, "3":3, "4":4, "4K":4, "5":5, "6":6, "6K":6, "7":7, "8": 8,"9":9,"9K":9, "9X":9}

def clean_ip_value(ip_str):
    if ip_str is np.nan:
        return np.nan
    result = ip_str.split(', ')[-1]
    if "/" in result:
        return result.split('/')[-1]
    return result

def split_ip_value(ip_str):
    if ip_str is np.nan or ip_str == "-":
        return np.nan, np.nan
    n = len(ip_str)
    ip_1,ip_2= ip_str[0:n//2], ip_str[n//2:]
    return IP_1[ip_1], IP_2[ip_2]


df1['Cleaned_IP'] = df1['IP Rating'].apply(lambda x: clean_ip_value(x))
df1['Cleaned_IP'] = df1['Cleaned_IP'].str.strip('IP')
df1['Cleaned_IP'].unique()

# df1[['IP_1', 'IP_2']] = df1['Cleaned_IP'].apply(lambda x: split_ip_value(x))
IP_tuple = df1['Cleaned_IP'].apply(lambda x: split_ip_value(x))
df1[['IP_1', 'IP_2']] = IP_tuple.apply(lambda x: pd.Series(x))
cols = df1.columns
cols =['Mfr Part Number', 'Mfr.', 'Pricing', 'RoHS', 'Lifecycle', 'IP Rating','Cleaned_IP', 'IP_1', 'IP_2',\
       'Product', 'Contact Gender', 'Contact_1',
       'Contact_2', 'Contact_3', 'Termination Style', 'UID']
df1[cols]

,Mfr Part Number,Mfr.,Pricing,RoHS,Lifecycle,IP Rating,Cleaned_IP,IP_1,IP_2,Product,Contact Gender,Contact_1,Contact_2,Contact_3,Termination Style,UID
0,"=""1404642""",Phoenix Contact,33.09,RoHS Compliant By Exemption,NaN,IP67,67,6.0,7.0,Connectors,Socket (Female),Socket (Female),NaN,NaN,Screw,1
1,MB7CKN0900-S1,Amphenol SINE Systems,25.36,RoHS Compliant,NaN,IP67,67,6.0,7.0,Connectors,Without Socket Contacts,Without Socket Contacts,NaN,NaN,Crimp,2
2,T 3636 001,Amphenol Tuchel,20.70,RoHS Compliant,NaN,IP40,40,4.0,0.0,Connectors,Socket (Female),Socket (Female),NaN,NaN,Solder,3
3,5-2271137-2,TE Connectivity,37.10,RoHS Compliant By Exemption,NaN,IP68,68,6.0,8.0,Connectors,Socket (Female),Socket (Female),NaN,NaN,Solder,4
4,"=""1441970""",Phoenix Contact,30.77,RoHS Compliant,NaN,IP67,67,6.0,7.0,Inserts,Socket (Female),Socket (Female),NaN,NaN,Through Hole,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8145,CL142001,Souriau,17.90,NaN,NaN,IP67,67,6.0,7.0,Accessories,-,NaN,NaN,NaN,-,142
8146,HAUTS0V128PRA300,Souriau,87.30,NaN,NaN,IP68,68,6.0,8.0,Plugs,NaN,NaN,NaN,NaN,NaN,3524
8147,JBXFH2C16MSSDSMR,Souriau,101.41,NaN,NaN,IP68,68,6.0,8.0,Connectors,Pin (Male),Pin (Male),NaN,NaN,Solder,115
8148,JBXEA1G10MSSDS,Souriau,32.88,NaN,NaN,IP40,40,4.0,0.0,Connectors,Pin (Male),Pin (Male),NaN,NaN,Solder,115


### Dropping of NA values
* Considered using replacing NA values with mode for categorical features, but feared about the class imbalance (given the %)
* Considered replacing NA values with median/mean for IP, but felt that it was unwise and only 10% of total data was missing for IP

In [1271]:
df1.isna().sum()

Mfr Part Number         0
Mfr.                    0
Pricing                 0
RoHS                 1367
Lifecycle            6356
IP Rating            1056
Product              1159
Contact Gender        882
Termination Style     595
UID                     0
Contact_1            1428
Contact_2            8093
Contact_3            8115
Cleaned_IP           1056
IP_1                 1612
IP_2                 1612
dtype: int64

In [1272]:
df1 = df1.drop(df1[df1['IP Rating'] == '-'].index)
df1 = df1.drop(df1[df1['Contact Gender'] == '-'].index)
df1 = df1.drop(df1[df1['Termination Style'] == '-'].index)

df1 = df1.dropna(subset = ['IP Rating'])
df1 = df1.dropna(subset = ['Contact Gender'])
df1 = df1.dropna(subset = ['Product'])
df1 = df1.dropna(subset = ['Termination Style'])

listToDrop = ['Mfr Part Number', 'IP Rating', 'Mfr.','RoHS','Lifecycle','Contact_1','Contact_2','Contact_3']
def conductDropColumns(toDrop, a):
    for column in toDrop:
        a.drop(column, axis =1, inplace=True)
conductDropColumns(listToDrop, df1)
df1
# print(df1.isna().sum())

,Pricing,Product,Contact Gender,Termination Style,UID,Cleaned_IP,IP_1,IP_2
0,33.09,Connectors,Socket (Female),Screw,1,67,6.0,7.0
1,25.36,Connectors,Without Socket Contacts,Crimp,2,67,6.0,7.0
2,20.70,Connectors,Socket (Female),Solder,3,40,4.0,0.0
3,37.10,Connectors,Socket (Female),Solder,4,68,6.0,8.0
4,30.77,Inserts,Socket (Female),Through Hole,5,67,6.0,7.0
...,...,...,...,...,...,...,...,...
8142,14.29,Connectors,Socket (Female),Crimp,142,40,4.0,0.0
8143,25.38,Connectors,Socket (Female),Crimp,142,68,6.0,8.0
8147,101.41,Connectors,Pin (Male),Solder,115,68,6.0,8.0
8148,32.88,Connectors,Pin (Male),Solder,115,40,4.0,0.0


#### Standardization of `Contact Gender` & `Termination Style` for vectorization
* Example: Pin (Male) -> pin male #Removed brackets, lower case
* Socket (Female), Pin (Male) -> socket female pin male #Removed commas


In [1273]:
def preprocess(text):
    # Remove non-alphanumeric characters and convert to lowercase
    if text is np.nan:
        return np.nan
    return ''.join(char.lower() for char in text if char.isalnum() or char.isspace())
df1['Cleaned_CG'] = df1['Contact Gender'].apply(lambda x: preprocess(x))
# (df1['Cleaned_CG'] == '').sum()
# df1['Contact Gender'].isna().sum()
df1['Cleaned_Termination_Style'] = df1['Termination Style'].apply(lambda x: preprocess(x))
conductDropColumns(['Contact Gender', 'Termination Style'], df1)


#### Vectorization using word2vec

In [1274]:
import spacy
nlp = spacy.load("en_core_web_sm")

def compute_average_embedding(text):
    # Tokenize text into words
    words = preprocess(text).split()

    # Initialize variables for storing word embeddings and counts
    embeddings =[]
    count = 0

    #Iterate through each word in the text
    for word in words:
        #Retrieve word embedding for the word
        embedding = nlp(word).vector
        embeddings.append(embedding)
        count += 1
    average_embedding = np.mean(embeddings,axis = 0)
    return average_embedding

df1['CG_Vector'] = df1['Cleaned_CG'].apply(lambda x: compute_average_embedding(x))
df1['TS_Vector'] = df1['Cleaned_Termination_Style'].apply(lambda x: compute_average_embedding(x))
df1['Product_Vector'] = df1['Product'].apply(lambda x: compute_average_embedding(x))

In [1275]:
df1


,Pricing,Product,UID,Cleaned_IP,IP_1,IP_2,Cleaned_CG,Cleaned_Termination_Style,CG_Vector,TS_Vector,Product_Vector
0,33.09,Connectors,1,67,6.0,7.0,socket female,screw,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[0.14882207, -0.17023766, -0.39692235, -0.6797...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
1,25.36,Connectors,2,67,6.0,7.0,without socket contacts,crimp,"[-0.835265, -0.10289645, -0.38677195, 0.029453...","[-1.3333964, -0.8013882, -0.23675042, -0.52361...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
2,20.70,Connectors,3,40,4.0,0.0,socket female,solder,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[-0.93961775, -0.6448832, 0.055333674, 0.98495...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
3,37.10,Connectors,4,68,6.0,8.0,socket female,solder,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[-0.93961775, -0.6448832, 0.055333674, 0.98495...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
4,30.77,Inserts,5,67,6.0,7.0,socket female,through hole,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[-0.77134824, -0.15877418, -0.39378506, -0.221...","[-1.4378164, 1.6269224, -1.223504, 1.0482403, ..."
...,...,...,...,...,...,...,...,...,...,...,...
8142,14.29,Connectors,142,40,4.0,0.0,socket female,crimp,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[-1.3333964, -0.8013882, -0.23675042, -0.52361...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
8143,25.38,Connectors,142,68,6.0,8.0,socket female,crimp,"[-1.4301493, -0.5614001, -0.41520047, 0.309323...","[-1.3333964, -0.8013882, -0.23675042, -0.52361...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
8147,101.41,Connectors,115,68,6.0,8.0,pin male,solder,"[-1.4162538, -0.9240855, -0.7708409, 0.8879362...","[-0.93961775, -0.6448832, 0.055333674, 0.98495...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."
8148,32.88,Connectors,115,40,4.0,0.0,pin male,solder,"[-1.4162538, -0.9240855, -0.7708409, 0.8879362...","[-0.93961775, -0.6448832, 0.055333674, 0.98495...","[-1.5307974, 0.601619, -0.9221842, 1.1769594, ..."


#### As I am unfamiliar with features as vectors, I decided to categorically encode the values of vectors

In [1276]:
def transform_feature_array(df, column_name):
    # Iterate over each element in the array
    for i in range(df[column_name].iloc[0].shape[0]):
        # Create new column names
        new_column_name = f"{column_name}_{i}"
        # Create new columns with the weights
        df[new_column_name] = df[column_name].apply(lambda x: x[i])
    # Drop the original column
    df.drop(columns=[column_name], inplace=True)
    return df
df_transformed = transform_feature_array(df1, 'CG_Vector')
df_transformed = transform_feature_array(df_transformed, 'TS_Vector')
df_transformed = transform_feature_array(df_transformed, 'Product_Vector')

/var/folders/2z/34n91y7j4x9fl0xccn0kh83r0000gq/T/ipykernel_88397/1585776271.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df[column_name].apply(lambda x: x[i])
/var/folders/2z/34n91y7j4x9fl0xccn0kh83r0000gq/T/ipykernel_88397/1585776271.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name] = df[column_name].apply(lambda x: x[i])
/var/folders/2z/34n91y7j4x9fl0xccn0kh83r0000gq/T/ipykernel_88397/1585776271.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [1277]:
df_transformed

,Pricing,Product,UID,Cleaned_IP,IP_1,IP_2,Cleaned_CG,Cleaned_Termination_Style,CG_Vector_0,CG_Vector_1,...,Product_Vector_86,Product_Vector_87,Product_Vector_88,Product_Vector_89,Product_Vector_90,Product_Vector_91,Product_Vector_92,Product_Vector_93,Product_Vector_94,Product_Vector_95
0,33.09,Connectors,1,67,6.0,7.0,socket female,screw,-1.430149,-0.561400,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
1,25.36,Connectors,2,67,6.0,7.0,without socket contacts,crimp,-0.835265,-0.102896,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
2,20.70,Connectors,3,40,4.0,0.0,socket female,solder,-1.430149,-0.561400,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
3,37.10,Connectors,4,68,6.0,8.0,socket female,solder,-1.430149,-0.561400,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
4,30.77,Inserts,5,67,6.0,7.0,socket female,through hole,-1.430149,-0.561400,...,-0.997423,0.861138,-0.322635,-0.223875,-0.758150,0.857184,1.996359,0.134829,-0.617082,0.875865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8142,14.29,Connectors,142,40,4.0,0.0,socket female,crimp,-1.430149,-0.561400,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
8143,25.38,Connectors,142,68,6.0,8.0,socket female,crimp,-1.430149,-0.561400,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
8147,101.41,Connectors,115,68,6.0,8.0,pin male,solder,-1.416254,-0.924085,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437
8148,32.88,Connectors,115,40,4.0,0.0,pin male,solder,-1.416254,-0.924085,...,-1.174257,0.486568,-0.460735,-0.856399,-0.987304,0.170135,2.729013,-0.396714,0.053473,1.017437


### Build and Train Model
Since IP Rating consists of two digits (the first digit representing dust protection and the second digit representing water protection), we have a few options regarding how to approach the modeling:
* Single Model Approach:
    * predict IP Rating as a whole, many possible classes represenitng all possible combinations of dust and water protection ratings
* Two-model Approach: (chosen for its flexibility, cons include doubled complexity and effort)
    * create two separate classification models, one for predicting the dust protection rating and another for predicting the water protection rating. 
    * train the dust protection model using features relevant to dust protection, and then train the water protection model using features relevant to water protection. 
    * use both models to predict the two digits of the IP rating.
* Two-stage

#### Training model on `IP_1` (Dust)

In [1278]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = df_transformed.columns
features = cols[0:1].append(cols[8:]) #Pricing + vectors

X = df_transformed[features]
y = df_transformed['IP_1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

# # Instantiate the model object
model = RandomForestClassifier(random_state=5, n_estimators=100)

# # Fit the model with the training data
model.fit(X_train, y_train)

# # predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# # Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)


Prediction on test data [6. 6. 6. ... 6. 6. 6.]

Accuracy_score on test dataset :  0.85973207249803


#### Training model on `IP_2` (Water)

In [1279]:
X = df_transformed[features]
y = df_transformed['IP_2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

# # Instantiate the model object
model = RandomForestClassifier(random_state=5, n_estimators=100)

# # Fit the model with the training data
model.fit(X_train, y_train)

# # predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# # Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)


Prediction on test data [7. 7. 9. ... 7. 5. 7.]

Accuracy_score on test dataset :  0.8006304176516943
